In [ ]:
import os
from MIDI_data_extractor import MIDI_data_extractor

In [ ]:
#go through bach folder and extract all midi files
midi_files = []
for root, dirs, files in os.walk("bach"):
    for file in files:
        if file.endswith(".mid"):
             midi_files.append(os.path.join(root, file))

In [ ]:
#extract all data from midi_files and put into list
midi_data = []
for file in midi_files:
    #print(file)
    midi_data.append(MIDI_data_extractor(file))
'''[note_on_note, note_on_velocity,
        control_change_control, control_change_value, program_change_program,
        end_marking, set_tempo_tempo,
        time_sig_num, itme_sig_den, time_sig_clocksperclick, time_sig_notated_32nd,
        key_sig(turn into numbers), [time], instrument_type, instrument_num, orig_instrument_type]'''